# Assignment 4
* Unit: SIT720
* Name: Bryon Baker
* Student #: 85031775

In [1]:
# TODO BEFORE SUBMISSION:
# Increase the train_test_iterations
# Uncomment the pip3 installs

In [2]:
#!pip3 install numpy==1.22.3
#!pip3 install pandas==1.4.2
#!pip3 install scikit-learn==1.0.2

In [3]:
import sys
import warnings
import random
import pandas as pd # dataframe manipulation
import numpy as np # linear algebra

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor

In [4]:
# Seed the random nunmber generator to ensure K-Means randomisation works properly.
random.seed(0)

# Constant used for train_test_split so there is predicatability across runs
global_random_state=11
train_test_iterations=10

print(f"Seeding random number generator. Random number is: {random.random()}")

Seeding random number generator. Random number is: 0.8444218515250481


# Helper Functions
This section defines any helper functions used in the code below

In [5]:
# Calculate the inverse of log(x+1) and assign any negative value to zero
# As described int he research paper, the inverse log transfor needs to take the inverse of y = ln(x+1). and if y < 0 then y  0
def inverse_log_transform( vals ):
    vals_x = np.exp(vals)-1
    vals_x[vals_x < 0] = 0
    
    return vals_x

# Standardise the dataset
Standardise all values in a dataset such that the mean = 0 and SD = 1

In [6]:
# A helper function to standardise all the values in the dataset with a mean=0 and SD=1
def standardise_dataset( dataset_name, dataset, column_headings ):
    # Standardise all attributes to zero mean and SD 1
    scaler = StandardScaler(copy=False, with_mean=True, with_std=True)
    scaler.fit(dataset)
    result = scaler.transform(dataset)
    # print(f"Transformed Data Frame:\n{SVM_STFWI_df}")
    m = np.mean(result, axis=0)
    s = np.std(result, axis=0)
    #print("Checking transformation")
    #print(f"Column  means:\n{m}")
    #print(f"Column  SD:\n{s}")

    # The StandardScaler converts to an array. Convert back to a DataFrame
    result_df = pd.DataFrame(result)
    result_df.columns = column_headings     # Reassign the column names.
    # print(result_df.head())
    
    return result_df

# Calculate the RMSE
Given two series (y and y-hat) take the inverse of a=ln(b+1) for each and replace any negative results with zero. Then calculate the root mean square error for the two series.

In [7]:
# Helper to calculate the RMSE
def calc_rmse( y, y_hat ):
    # Perform the inverse transform of y and h_hat
    yy = np.exp(y)-1
    yy[yy < 0] = 0
    
    yy_hat = np.exp(y_hat)-1
    yy_hat[yy_hat < 0] = 0
        
    rmse = mean_squared_error(yy, yy_hat, squared=False)
    
    return rmse

# Save Datasets
This helper creates a local copy of the post-processed datasets.
The function has been commented out for final assignment submission in case the local filesystem write causes the notebook to crash and get a zero mark.

In [8]:
from pathlib import Path 


def save_dataset( filename, dataset ):
    return   # This is bypassed for the final assignment submission in case this causes the notbook to crash on a different machine
    file_dir = "~/datasets/SIT720/Ass4/transformed/"
    filepath = Path(file_dir)  
    filepath.parent.mkdir(parents=True, exist_ok=True)
    
    full_filename = file_dir + filename + ".csv"
    dataset.to_csv(full_filename)

# Load the source dataset
Dataset is stored in public git repo. COnfigure the ssl context in order to access github raw data files.

In [9]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

url = 'https://raw.githubusercontent.com/bryonbaker/datasets/main/SIT720/Ass4/forestfires.csv'
full_df = pd.read_csv(url)
print(f"{full_df.head()}\n")

   X  Y month  day  FFMC   DMC     DC  ISI  temp  RH  wind  rain  area
0  7  5   mar  fri  86.2  26.2   94.3  5.1   8.2  51   6.7   0.0   0.0
1  7  4   oct  tue  90.6  35.4  669.1  6.7  18.0  33   0.9   0.0   0.0
2  7  4   oct  sat  90.6  43.7  686.9  6.7  14.6  33   1.3   0.0   0.0
3  8  6   mar  fri  91.7  33.3   77.5  9.0   8.3  97   4.0   0.2   0.0
4  8  6   mar  sun  89.3  51.3  102.2  9.6  11.4  99   1.8   0.0   0.0



# Question 1

Read the article and reproduce the RMSE results presented in Table 3 using Python modules and packages (including your own script or customised codes). Write a report summarising the dataset, used ML methods, experiment protocol and results including variations, if any. During reproducing the results:               (15 Marks)

1. you should use the same set of features used by the authors.
2. you should use the same classifier with exact parameter values.
3. you should use the same training/test splitting approach as used by the authors.
4. you should use the same pre/post processing, if any, used by the authors.
5. you should report the same performance metric (RMSE) as shown in Table 3.

# Pre Process the Data
1. One hot encode the day and month attributes.
2. Ordinal encode the day/month for DT, MR, RF
3. Transform the area to be y = ln(area+1)

## Preprocess area
Reduce the skewness of area as per procedure on page 5.
This will be used to train the model, but it is important to take the inverse log after prediction so as to find the predicted area. 


In [10]:
# Check the dataset has no NaNs.
if full_df.isnull().sum().sum() == 0:
    print("Data does not have any missing values.")
else:
    print("Data has missing values and needs pre processing.")
    print("Stopping....")
    quit() 

Data does not have any missing values.


In [11]:
# Create a new area feature and calculate the values as new-area = ln(area+1)
area_df = pd.DataFrame(full_df["area"])
area_df["xfrmd_area"] = full_df["area"] +1                 # Create a new area column that will contain the ln(area+1)
print(area_df.head())
area_df["xfrmd_area"] = np.log(area_df['xfrmd_area'])      # Take the ln(area+1)
print(area_df.head()) 

   area  xfrmd_area
0   0.0         1.0
1   0.0         1.0
2   0.0         1.0
3   0.0         1.0
4   0.0         1.0
   area  xfrmd_area
0   0.0         0.0
1   0.0         0.0
2   0.0         0.0
3   0.0         0.0
4   0.0         0.0


### 1-of-C Encoding

Perform 1-of-C encoding on the day and month features. These will be used tor SVM, NN, and MR.

In [12]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(handle_unknown='ignore', drop=None, dtype='int')

In [13]:
# One-hot encode month
transformed = ohe.fit_transform(full_df[['month']])
ohe_month_df = pd.DataFrame(transformed.toarray())
ohe_month_df.columns = ohe.categories_[0]

In [14]:
# One hot encode day
transformed = ohe.fit_transform(full_df[['day']])
ohe_day_df = pd.DataFrame(transformed.toarray())
ohe_day_df.columns = ohe.categories_[0]

In [15]:
# encoded_df holds all the training.test data that has been 1-of-C encoded.
ohe_encoded_df = full_df.drop(['month','day','area'], axis=1)
ohe_encoded_df = pd.concat([ohe_encoded_df, ohe_month_df], axis=1)
ohe_encoded_df = pd.concat([ohe_encoded_df, ohe_day_df], axis=1)
ohe_encoded_df.head(20)

,X,Y,FFMC,DMC,DC,ISI,temp,RH,wind,rain,...,nov,oct,sep,fri,mon,sat,sun,thu,tue,wed
0,7,5,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,...,0,0,0,1,0,0,0,0,0,0
1,7,4,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,...,0,1,0,0,0,0,0,0,1,0
2,7,4,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,...,0,1,0,0,0,1,0,0,0,0
3,8,6,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,...,0,0,0,1,0,0,0,0,0,0
4,8,6,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,...,0,0,0,0,0,0,1,0,0,0
5,8,6,92.3,85.3,488.0,14.7,22.2,29,5.4,0.0,...,0,0,0,0,0,0,1,0,0,0
6,8,6,92.3,88.9,495.6,8.5,24.1,27,3.1,0.0,...,0,0,0,0,1,0,0,0,0,0
7,8,6,91.5,145.4,608.2,10.7,8.0,86,2.2,0.0,...,0,0,0,0,1,0,0,0,0,0
8,8,6,91.0,129.5,692.6,7.0,13.1,63,5.4,0.0,...,0,0,1,0,0,0,0,0,1,0
9,7,5,92.5,88.0,698.6,7.1,22.8,40,4.0,0.0,...,0,0,1,0,0,1,0,0,0,0


### Ordinal Encoding

Perform ordinal encoding of day and month. These will be used for RF, and DT models

In [16]:
# Ordinal encode month
from sklearn.preprocessing import OrdinalEncoder

ore = OrdinalEncoder(dtype='int')

transformed = ore.fit_transform(full_df[['month']])
ord_month_df = pd.DataFrame(transformed)
ord_month_df.columns = ['month']

In [17]:
# Ordinal encode month
from sklearn.preprocessing import OrdinalEncoder

ore = OrdinalEncoder(dtype='int')

transformed = ore.fit_transform(full_df[['day']])
ord_day_df = pd.DataFrame(transformed)
ord_day_df.columns = ['day']

In [18]:
ord_encoded_df = full_df     # RF and DT need the ordinal encoded dataframe
ord_encoded_df = full_df.drop(['month','day','area'], axis=1)
ord_encoded_df = pd.concat([ord_encoded_df, ord_month_df], axis=1)
ord_encoded_df = pd.concat([ord_encoded_df, ord_day_df], axis=1)
ord_encoded_df.head(20)

,X,Y,FFMC,DMC,DC,ISI,temp,RH,wind,rain,month,day
0,7,5,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,7,0
1,7,4,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,10,5
2,7,4,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,10,2
3,8,6,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,7,0
4,8,6,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,7,3
5,8,6,92.3,85.3,488.0,14.7,22.2,29,5.4,0.0,1,3
6,8,6,92.3,88.9,495.6,8.5,24.1,27,3.1,0.0,1,1
7,8,6,91.5,145.4,608.2,10.7,8.0,86,2.2,0.0,1,1
8,8,6,91.0,129.5,692.6,7.0,13.1,63,5.4,0.0,11,5
9,7,5,92.5,88.0,698.6,7.1,22.8,40,4.0,0.0,11,2


In [19]:
del full_df         # Make sure we don't accidentally use full_df later

## Create Sub-Datasets

Now we have all the raw ingredients for all datasets, create the sub datasets that the different models will need.

Although some of the sataset requirements are the same across models, the approach taken in this assignment is to have four datasets per model. This approach was chosen just in case there was a need to further pre-process the datasets uniquely for a given model.

In [20]:
# Define the sub datasets from the transformed dataset

# Get the 1-of-C encoded column names for month and day
month_cols = ohe_month_df.columns.tolist()
day_cols = ohe_day_df.columns.tolist()

# Construct the dataframe column names
STFWI_ohe_encoded_cols = ['X','Y','FFMC','DMC','DC','ISI']
STFWI_ohe_encoded_cols = STFWI_ohe_encoded_cols + month_cols + day_cols
STM_ohe_encoded_cols = ['X','Y','temp','RH','wind','rain']
STM_ohe_encoded_cols = STM_ohe_encoded_cols + month_cols + day_cols
FWI_ohe_encoded_cols = ['FFMC','DMC','DC','ISI']
M_ohe_encoded_cols = ['temp','RH','wind','rain']

# Construct the dataframe column names
STFWI_ord_encoded_cols = ['X','Y','month','day','FFMC','DMC','DC','ISI']
STM_ord_encoded_cols = ['X','Y','month','day','temp','RH','wind','rain']
FWI_ord_encoded_cols = ['FFMC','DMC','DC','ISI']
M_ord_encoded_cols = ['temp','RH','wind','rain']


In [21]:
# create feature tables for MR, SVM, and NN
STFWI_ohe_encoded_df = ohe_encoded_df[STFWI_ohe_encoded_cols]
STM_ohe_encoded_df = ohe_encoded_df[STM_ohe_encoded_cols]
FWI_ohe_encoded_df = ohe_encoded_df[FWI_ohe_encoded_cols]
M_ohe_encoded_df = ohe_encoded_df[M_ohe_encoded_cols]

# Create the feature tables for RF and DT
STFWI_ord_encoded_df = ord_encoded_df[STFWI_ord_encoded_cols]
STM_ord_encoded_df = ord_encoded_df[STM_ord_encoded_cols]
FWI_ord_encoded_df = ord_encoded_df[FWI_ord_encoded_cols]
M_ord_encoded_df = ord_encoded_df[M_ord_encoded_cols]

# Create the transformed area ln(area+1)
y_xformed = area_df["xfrmd_area"]
y_vanilla = area_df["area"]

In [22]:
# Display which columns are used for which datasets.
print("SVM, MR, and NN Dataset columns:")
print("="*16)
print(f"STFWI Dataset columns:\n{STFWI_ohe_encoded_df.columns}\n")
print(f"STM Dataset columns:\n{STM_ohe_encoded_df.columns}\n")
print(f"FWI Dataset columns:\n{FWI_ohe_encoded_df.columns}\n")
print(f"M Dataset columns:\n{M_ohe_encoded_df.columns}\n")
print("="*16)
print("DT and RF Dataset columns:")
print("="*16)
print(f"STFWI Dataset columns:\n{STFWI_ord_encoded_df.columns}\n")
print(f"STM Dataset columns:\n{STM_ord_encoded_df.columns}\n")
print(f"FWI Dataset columns:\n{FWI_ord_encoded_df.columns}\n")
print(f"M Dataset columns:\n{M_ord_encoded_df.columns}\n")

SVM, MR, and NN Dataset columns:
STFWI Dataset columns:
Index(['X', 'Y', 'FFMC', 'DMC', 'DC', 'ISI', 'apr', 'aug', 'dec', 'feb', 'jan',
       'jul', 'jun', 'mar', 'may', 'nov', 'oct', 'sep', 'fri', 'mon', 'sat',
       'sun', 'thu', 'tue', 'wed'],
      dtype='object')

STM Dataset columns:
Index(['X', 'Y', 'temp', 'RH', 'wind', 'rain', 'apr', 'aug', 'dec', 'feb',
       'jan', 'jul', 'jun', 'mar', 'may', 'nov', 'oct', 'sep', 'fri', 'mon',
       'sat', 'sun', 'thu', 'tue', 'wed'],
      dtype='object')

FWI Dataset columns:
Index(['FFMC', 'DMC', 'DC', 'ISI'], dtype='object')

M Dataset columns:
Index(['temp', 'RH', 'wind', 'rain'], dtype='object')

DT and RF Dataset columns:
STFWI Dataset columns:
Index(['X', 'Y', 'month', 'day', 'FFMC', 'DMC', 'DC', 'ISI'], dtype='object')

STM Dataset columns:
Index(['X', 'Y', 'month', 'day', 'temp', 'RH', 'wind', 'rain'], dtype='object')

FWI Dataset columns:
Index(['FFMC', 'DMC', 'DC', 'ISI'], dtype='object')

M Dataset columns:
Index(['temp', 'R

## Decision Tree Dataset Preparation

In [23]:
DT_STFWI_df = STFWI_ord_encoded_df
DT_STM_df = STM_ord_encoded_df
DT_FWI_df = FWI_ord_encoded_df
DT_M_df = M_ord_encoded_df

save_dataset( "DT_STFWI_df", DT_STFWI_df )
save_dataset( "DT_STM_df", DT_STM_df )
save_dataset( "DT_FWI_df", DT_FWI_df )
save_dataset( "DT_M_df", DT_M_df )



## Random Forrest Dataset Preparation

In [24]:
RF_STFWI_df = STFWI_ord_encoded_df
RF_STM_df = STM_ord_encoded_df
RF_FWI_df = FWI_ord_encoded_df
RF_M_df = M_ord_encoded_df

# Save the dataset to a file for manual exploration and verifiaction of feature engineering.
save_dataset( "RF_STFWI_df", RF_STFWI_df )
save_dataset( "RF_STM_df", RF_STM_df )
save_dataset( "RF_FWI_df", RF_FWI_df )
save_dataset( "RF_M_df", RF_M_df )

## SVM Data Set Preparation

In [25]:
# Standardise all attributes to zero mean and SD 1
SVM_STFWI_df = STFWI_ohe_encoded_df
SVM_STM_df = STM_ohe_encoded_df
SVM_FWI_df = FWI_ohe_encoded_df
SVM_M_df = M_ohe_encoded_df

SVM_STFWI_df = standardise_dataset("SVM_STFWI_df", SVM_STFWI_df, STFWI_ohe_encoded_cols)
SVM_STM_df = standardise_dataset("SVM_STM_df", SVM_STM_df, STM_ohe_encoded_cols)
SVM_FWI_df = standardise_dataset("SVM_FWI_df", SVM_FWI_df, FWI_ohe_encoded_cols)
SVM_M_df = standardise_dataset("SVM_M_df", SVM_M_df, M_ohe_encoded_cols)

# Save the dataset to a file for manual exploration and verifiaction of feature engineering.
save_dataset( "SVM_STFWI_df", SVM_STFWI_df )
save_dataset( "SVM_STM_df", SVM_STM_df )
save_dataset( "SVM_FWI_df", SVM_FWI_df )
save_dataset( "SVM_M_df", SVM_M_df )

## Multiple Rgression Data Set Preparation

In [26]:
MR_STFWI_df = STFWI_ohe_encoded_df
MR_STM_df = STM_ohe_encoded_df
MR_FWI_df = FWI_ohe_encoded_df
MR_M_df = M_ohe_encoded_df

MR_STFWI_df = standardise_dataset("MR_STFWI_df", MR_STFWI_df, STFWI_ohe_encoded_cols)
MR_STM_df = standardise_dataset("MR_STM_df", MR_STM_df, STM_ohe_encoded_cols)
MR_FWI_df = standardise_dataset("MR_FWI_df", MR_FWI_df, FWI_ohe_encoded_cols)
MR_M_df = standardise_dataset("MR_M_df", MR_M_df, M_ohe_encoded_cols)

# Save the dataset to a file for manual exploration and verifiaction of feature engineering.
save_dataset( "MR_STFWI_df", MR_STFWI_df )
save_dataset( "MR_STM_df", MR_STM_df )
save_dataset( "MR_FWI_df", MR_FWI_df )
save_dataset( "MR_M_df", MR_M_df )

# Create the Results DataFrame

This dataframe will contain all of the results for the assignment.

In [27]:
results_df = pd.DataFrame(columns=['Model','STFWI (RMSE)','STM (RMSE)','FWI (RMSE)','M (RMSE)'])

# Calculate the Naive results

In [28]:
# Generate an array with the predicted values being the mean of the area
y = area_df["area"]
y_pred = np.mean(y)
y_naive_pred = np.repeat(y_pred, len(y))

# Calculate the naive RMSE
naive_rmse = mean_squared_error(y, y_naive_pred, squared=False)
#print(naive_rmse)

# Add the naive results to the Results Table
rmse_df = pd.DataFrame([['Naive', naive_rmse, naive_rmse, naive_rmse, naive_rmse]], columns=['Model','STFWI (RMSE)','STM (RMSE)','FWI (RMSE)','M (RMSE)'])
#results_df = results_df.append(rmse_df)
results_df = pd.concat([results_df, rmse_df])

In [29]:
results_df.head()

,Model,STFWI (RMSE),STM (RMSE),FWI (RMSE),M (RMSE)
0,Naive,63.594226,63.594226,63.594226,63.594226


# Decision Tree

Scoring details:

RMSE: https://scikit-learn.org/stable/modules/model_evaluation.html

https://stackoverflow.com/questions/62514395/score-obtained-from-cross-val-score-is-rmse-or-mse


In [30]:
from sklearn.model_selection import train_test_split

print("DECISION TREE")
print("*"*30)

# Get the list of training / validating datasets
train_set = { "STFWI" : DT_STFWI_df, "STM" : DT_STM_df, "FWI" : DT_FWI_df, "M" : DT_M_df}

rmse_score = []

print(f"Train/Test running for {train_test_iterations} iterations.")

# Iterate over each dataset, fit the mode and evaluate the performance with RMSE
for key in train_set:
    print("="*10)
    print(key)

    # Run 10 train / test splits and take the average RMSE for each dataset
    results = []
    for run in range(train_test_iterations):
        # Split the dataset for train/test
        X_train, X_test, y_train, y_test = train_test_split( train_set[key], y_xformed, test_size=0.10, random_state=8)

        # Set hyperparameters according to the research paper
        # min_samples_split = 2
        model = DecisionTreeRegressor(random_state=1, min_samples_split = 2)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        rmse = calc_rmse( y_pred, y_test )
        results.append(rmse)

    avg = sum(results) / len(results)
    rmse_score.append(round(avg,1))
    # print(results)
    print(f"RMSE: {avg}")

rmse_df = pd.DataFrame([['DT', rmse_score[0], rmse_score[1], rmse_score[2], rmse_score[3]]], columns=['Model','STFWI (RMSE)','STM (RMSE)','FWI (RMSE)','M (RMSE)'])
#results_df = results_df.append(rmse_df)
results_df = pd.concat([results_df, rmse_df])
    
del rmse_score, train_set

DECISION TREE
******************************
Train/Test running for 10 iterations.
STFWI
RMSE: 33.36797772613939
STM
RMSE: 44.667739854613714
FWI
RMSE: 33.079516673420585
M
RMSE: 56.09609903900008


# Random Forrest

In [31]:
from sklearn.ensemble import RandomForestRegressor

print("RANDOM FORREST")
print("*"*30)


# Get the list of training / validating datasets
train_set = { "STFWI" : RF_STFWI_df, "STM" : RF_STM_df, "FWI" : RF_FWI_df, "M" : RF_M_df}

rmse_score = []

print(f"Train/Test running for {train_test_iterations} iterations.")

for key in train_set:
    print(key)
    print("="*10)
    
    # Run 10 train / test splits and take the average RMSE for each dataset
    results = []
    for run in range(train_test_iterations):
        # Split the dataset for train/test
        X_train, X_test, y_train, y_test = train_test_split( train_set[key], y_xformed, test_size=0.10, random_state=8)

        # Set hyperparameters according to research paper
        # Number of trees: 500
        model = RandomForestRegressor(random_state=1, n_estimators = 500)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        rmse = calc_rmse( y_pred, y_test )
        results.append(rmse)
    
    avg = sum(results) / len(results)
    rmse_score.append(round(avg,1))
    # print(results)
    print(f"RMSE: {avg}")

    
rmse_df = pd.DataFrame([['RF', rmse_score[0], rmse_score[1], rmse_score[2], rmse_score[3]]], columns=['Model','STFWI (RMSE)','STM (RMSE)','FWI (RMSE)','M (RMSE)'])
#results_df = results_df.append(rmse_df)
results_df = pd.concat([results_df, rmse_df])

del rmse_score, train_set, rmse_df     # Delete the variables you do not want to be picked up accidentally elsewhere.

RANDOM FORREST
******************************
Train/Test running for 10 iterations.
STFWI
RMSE: 32.699069995938146
STM
RMSE: 32.40763807823598
FWI
RMSE: 32.721179350330246
M
RMSE: 32.36444927022474


# Multiple Regression

In [32]:
from sklearn.linear_model import LinearRegression

print("MULTIPLE REGRESSION")
print("*"*30)

# Get the list of training / validating datasets
train_set = { "STFWI" : MR_STFWI_df, "STM" : MR_STM_df, "FWI" : MR_FWI_df, "M" : MR_M_df}

print(f"Train/Test running for {train_test_iterations} iterations.")

rmse_score = []

for key in train_set:
    print(key)
    print("="*10)

    # Run 10 train / test splits and take the average RMSE for each dataset
    results = []
    for run in range(train_test_iterations):    
        # Split the dataset for train/test
        X_train, X_test, y_train, y_test = train_test_split( train_set[key], y_xformed, test_size=0.10, random_state=global_random_state)

        # Set hyperparameters according to research paper.
        # Use Least squares method.  Nothing to do. From scikit documentation: 
        # From the implementation point of view, this is just plain Ordinary Least Squares (scipy.linalg.lstsq)"
        model = LinearRegression()
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        rmse = calc_rmse( y_pred, y_test )
        results.append(rmse)
    
    avg = sum(results) / len(results)
    rmse_score.append(round(avg,1))
    # print(results)
    print(f"RMSE: {avg}")


rmse_df = pd.DataFrame([['MR', rmse_score[0], rmse_score[1], rmse_score[2], rmse_score[3]]], columns=['Model','STFWI (RMSE)','STM (RMSE)','FWI (RMSE)','M (RMSE)'])
results_df = pd.concat([results_df, rmse_df])
#results_df = results_df.append(rmse_df)

del rmse_score, train_set, rmse_df     # Delete the variables you do not want to be picked up accidentally elsewhere.

MULTIPLE REGRESSION
******************************
Train/Test running for 10 iterations.
STFWI
RMSE: 43.53604830037587
STM
RMSE: 43.40038262035092
FWI
RMSE: 43.39933286728688
M
RMSE: 43.38933684545374


# SVM

In [33]:
from sklearn.svm import SVR

print("SVM")
print("*"*30)

# Get the list of training / validating datasets
train_set = { "STFWI" : SVM_STFWI_df, "STM" : SVM_STM_df, "FWI" : SVM_FWI_df, "M" : SVM_M_df}
# The hyperparameters to use for each dataset
gamma = { "STFWI" : 2.0e-5 , "STM" : 2.0e-3, "FWI" : 2.0e-3, "M" : 2.0e-2}

print(f"Train/Test running for {train_test_iterations} iterations.")

rmse_score = []

for key in train_set:
    print(train_test_iterations)
    print("="*10)

    # Run 10 train / test splits and take the average RMSE for each dataset
    results = []
    for run in range(train_test_iterations):        
        # Split the dataset for train/test
        X_train, X_test, y_train, y_test = train_test_split( train_set[key], y_xformed, test_size=0.10, random_state=global_random_state)

        # Set hyperparameters according to research paper.
        model = SVR(gamma=gamma[key], C=3, epsilon=0.1)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        rmse = calc_rmse( y_pred, y_test )
        results.append(rmse)
    
    avg = sum(results) / len(results)
    rmse_score.append(round(avg,1))
    # print(results)
    print(f"RMSE: {avg}")

rmse_df = pd.DataFrame([['SVM', rmse_score[0], rmse_score[1], rmse_score[2], rmse_score[3]]], columns=['Model','STFWI (RMSE)','STM (RMSE)','FWI (RMSE)','M (RMSE)'])
results_df = pd.concat([results_df, rmse_df])
#results_df = results_df.append(rmse_df)

del rmse_score, train_set, rmse_df     # Delete the variables you do not want to be picked up accidentally elsewhere.

SVM
******************************
Train/Test running for 10 iterations.
10
RMSE: 43.8059800161267
10
RMSE: 43.735354570974934
10
RMSE: 43.79721785353759
10
RMSE: 43.84035512327204


# Display Final Results

In [34]:
results_df.round(1)

,Model,STFWI (RMSE),STM (RMSE),FWI (RMSE),M (RMSE)
0,Naive,63.594226,63.594226,63.594226,63.594226
0,DT,33.4,44.7,33.1,56.1
0,RF,32.7,32.4,32.7,32.4
0,MR,43.5,43.4,43.4,43.4
0,SVM,43.8,43.7,43.8,43.8


# Question 2

Design and develop your own ML solution for this problem. The proposed solution should be different from all approaches mentioned in the provided article. This does not mean that you must have to choose a new ML algorithm. You can develop a novel solution by changing the feature selection approach or parameter optimisations process of used ML methods or using different ML methods or adding regularization or different combinations of them. This means, the proposed system should be substantially different from the methods presented in the article but not limited to only change of ML methods. Compare the RMSE result with reported methods in the article. Write in your report summarising your solution design and outcomes. The report should include:

1. Motivation behind the proposed solution.
2. How the proposed solution is different from existing ones.
3. Detail description of the model including all parameters so that any reader can implement your model.
4. Description of experimental protocol.
5. Evaluation metrics. 
6. Present results using tables and graphs. 
7. Compare and discuss results with respect to existing literatures.
8. Appropriate references (IEEE numbered).


## Prepare the Neural Networkk Dataset

In [35]:
NN_STFWI_df = STFWI_ohe_encoded_df
NN_STM_df = STM_ohe_encoded_df
NN_FWI_df = FWI_ohe_encoded_df
NN_M_df = M_ohe_encoded_df

NN_STFWI_df = standardise_dataset("NN_STFWI_df", NN_STFWI_df, STFWI_ohe_encoded_cols)
NN_STM_df = standardise_dataset("NN_STM_df", NN_STM_df, STM_ohe_encoded_cols)
NN_FWI_df = standardise_dataset("NN_FWI_df", NN_FWI_df, FWI_ohe_encoded_cols)
NN_M_df = standardise_dataset("NN_M_df", NN_M_df, M_ohe_encoded_cols)

# Save the dataset to a file for manual exploration and verifiaction of feature engineering.
save_dataset( "NN_STFWI_df", NN_STFWI_df )
save_dataset( "NN_STM_df", NN_STM_df )
save_dataset( "NN_FWI_df", NN_FWI_df )
save_dataset( "NN_M_df", NN_M_df )

## Neural Network Regressor

# Find the best Neural Network Model

Perform a grid search to find the best hyperparameters. We will start with the parameters described in the research paper and add additional hyperparameters.

The method will use an inner/outer train/test approach. The dataset is divided using 10-fold, and each fold is split into 3 for conducting the grid search.

The main hyperparameter to explor is the number of hidden dimensions/layer size. A different gradient descent approach will be explored a,d the strength of the regularisation. Two solvers were selected out of a possible three. adam was not explored because this is recommended for large datasets.

## NOTE
This code takes approximately two hours to run so it has been tagged not to run by default. To run this block of code change the block type from Raw to Code. Refer to the assignment-submission document for a copy of the grid search output.

## Train the Neural Network

Using th hyperparameters identified in the grid search above, train the model and measure the performance. Append the results to the main results table.

In [39]:
from sklearn.neural_network import MLPRegressor

print("NEURAL NETWORK")
print("*"*30)

# Get the list of training / validating datasets
train_set = { "STFWI" : NN_STFWI_df, "STM" : NN_STM_df, "FWI" : NN_FWI_df, "M" : NN_M_df}

# The hyperparameters to use for each dataset. These are identified from the grid search above.
H = { "STFWI" : 2 , "STM" : 2, "FWI" : 6, "M" : 6}
alpha = { "STFWI" : 0.05 , "STM" : 0.05, "FWI" : 0.05, "M" : 0.05}
solver = { "STFWI" : 'sgd' , "STM" : 'sgd', "FWI" : 'sgd', "M" : 'sgd'}
learning_rate = { "STFWI" : 'constant' , "STM" : 'constant', "FWI" : 'constant', "M" : 'adaptive'}

rmse_score = []

# Iterate over each dataset, fit the mode and evaluate the performance with RMSE
for key in train_set:
    print("="*10)
    print(key)

    # Run 10 train / test splits and take the average RMSE for each dataset
    results = []
    for run in range(10):
        # Split the dataset for train/test
        X_train, X_test, y_train, y_test = train_test_split( train_set[key], y_xformed, test_size=0.10, random_state=8)

        # Set hyperparameters according to the research paper
        model = MLPRegressor(random_state=1, max_iter=20000, activation='tanh', hidden_layer_sizes=(H[key]), solver=solver[key], alpha=alpha[key], learning_rate=learning_rate[key])
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)            
        rmse = calc_rmse( y_pred, y_test )
        results.append(rmse)

        # Calculate the rmse on the training dataset. If rmse(train) < rmse(test) then the model is overfitten.
        train_pred = model.predict(X_train)
        train_rmse = calc_rmse( train_pred, y_train)
        print(train_rmse)

    avg = sum(results) / len(results)
    rmse_score.append(round(avg,1))
    # print(results)
    print(f"RMSE: {avg}")

rmse_df = pd.DataFrame([['Q2: NN', rmse_score[0], rmse_score[1], rmse_score[2], rmse_score[3]]], columns=['Model','STFWI (RMSE)','STM (RMSE)','FWI (RMSE)','M (RMSE)'])
results_df = pd.concat([results_df, rmse_df])
#results_df = results_df.append(rmse_df)
    
del rmse_score, train_set

NEURAL NETWORK
******************************
STFWI
67.09553097750889
67.09553097750889
67.09553097750889
67.09553097750889
67.09553097750889
67.09553097750889
67.09553097750889
67.09553097750889
67.09553097750889
67.09553097750889
RMSE: 32.4667033491962
STM
67.05048692008042
67.05048692008042
67.05048692008042
67.05048692008042
67.05048692008042
67.05048692008042
67.05048692008042
67.05048692008042
67.05048692008042
67.05048692008042
RMSE: 32.438487157969625
FWI
67.13285314478401
67.13285314478401
67.13285314478401
67.13285314478401
67.13285314478401
67.13285314478401
67.13285314478401
67.13285314478401
67.13285314478401
67.13285314478401
RMSE: 32.38758030377324
M
67.09459880158919
67.09459880158919
67.09459880158919
67.09459880158919
67.09459880158919
67.09459880158919
67.09459880158919
67.09459880158919
67.09459880158919
67.09459880158919
RMSE: 32.542732657400045


## Update Results Table

In [37]:
results_df.round(1)

,Model,STFWI (RMSE),STM (RMSE),FWI (RMSE),M (RMSE)
0,Naive,63.594226,63.594226,63.594226,63.594226
0,DT,33.4,44.7,33.1,56.1
0,RF,32.7,32.4,32.7,32.4
0,MR,43.5,43.4,43.4,43.4
0,SVM,43.8,43.7,43.8,43.8
0,Q2: NN,32.5,32.4,32.4,32.5
